In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

######Reference  https://www.kaggle.com/code/ambrosm/tpsmay22-eda-which-makes-sense

https://colab.research.google.com/drive/1d8ckS5wJkZzrLmtrrcMK284d5wczcVd8#scrollTo=2HgSxukmyeY3


In [ ]:
# Tabular play ground series May.
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMClassifier
import warnings, gc, string, random
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff


In [ ]:
print(pd.__version__)

In [ ]:
# Train data
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv', sep=',', index_col='id')
train1=train.copy()       #Copy training data in case we need original data later.
train.shape

In [ ]:
# Test data
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv',sep=',',index_col='id')
test.head()

In [ ]:
#submission data
submit=pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')


In [ ]:

# Check data
train.info()
# It seems all except f_27 are float type . F_27 is object string type. Also there are no missing values in any columns.

In [ ]:
train.describe()

In [ ]:
# Check column names and types
print(' Column names are : {}'.format(train.columns))

print(' Column types are : {}'.format(train.dtypes))

In [ ]:
train.select_dtypes(['float64']).head()

In [ ]:
#Convert float to int for memory saving and fast execution
float_list_train = train.select_dtypes(include=[np.float64]).columns
#train[float_list_train] = train[float_list_train].astype(np.float32)
int_list_train = train.select_dtypes(include=[np.int64]).columns
#train[int_list_train] = train[int_list_train].astype(np.int16)
#train.info()

In [ ]:
#Convert float to int for memory saving and fast execution for test data also
float_list_test = test.select_dtypes(include=[np.float64]).columns
#float_list_test
#test[float_list_test] = test[float_list_test].astype(np.float32)
int_list_test = test.select_dtypes(include=[np.int64]).columns
#test[int_list_test] = test[int_list_test].astype(np.int16)
#test.info()

In [ ]:
# Check out distribution of features using diagram
# Check float features

fig, axs = plt.subplots(4,4 ,figsize=(16,16))
for f, ax in zip(float_list_train,axs.ravel()):
  ax.hist(train[f], density=True, bins=100)
  ax.set_title(f'Train {f}, std={train[f].std():.1f}')
plt.suptitle('Histograms of the float features',y=0.93,fontsize=20)
plt.show()


In [ ]:
# Correlation matrix of float features
plt.figure(figsize=(16,16))
sns.heatmap(train[float_list_train].corr(),center=0,annot=True,fmt='.3f')

In [ ]:
# Plot dependence between every feature and the target
def plot_mutual_info_diagram(df, features, ncols=4, by_quantile=True, mutual_info=True,
                             title='How the target probability depends on single features'):
    def H(p):
        """Entropy of a binary random variable in nat"""
        return -np.log(p) * p - np.log(1-p) * (1-p)
                 
    nrows = (len(features) + ncols - 1) // ncols
    fig, axs = plt.subplots(nrows, ncols, figsize=(16, nrows*4), sharey=True)
    for f, ax in zip(features, axs.ravel()):
        temp = pd.DataFrame({f: df[f].values,
                             'state': df.target.values})
        temp = temp.sort_values(f)
        temp.reset_index(inplace=True)
        rolling_mean = temp.state.rolling(15000, center=True, min_periods=1).mean()
        if by_quantile:
            ax.scatter(temp.index, rolling_mean, s=2)
        else:
            ax.scatter(temp[f], rolling_mean, s=2)
        if mutual_info and by_quantile:
            ax.set_xlabel(f'{f} mi={H(temp.state.mean()) - H(rolling_mean[~rolling_mean.isna()].values).mean():.5f}')
        else:
            ax.set_xlabel(f'{f}')
    plt.suptitle(title, y=0.90, fontsize=20)
    plt.show()

plot_mutual_info_diagram(train, float_list_train,
                         title='How the target probability depends on the float features')

In [ ]:
#Check integerfeatures
# Training histograms

from matplotlib.ticker import MaxNLocator

figure = plt.figure(figsize=(16, 16))

for i, f in enumerate(int_list_train):
    plt.subplot(4, 4, i+1)
    ax = plt.gca()
    vc = train[f].value_counts()
    ax.bar(vc.index, vc)
    
    ax.set_xlabel(f'Train {f}')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True)) # only integer labels
plt.suptitle('Histograms of the integer features', y=1.0, fontsize=20)
figure.tight_layout(h_pad=1.0)
plt.show()


In [ ]:
plot_mutual_info_diagram(train, int_list_train,
                         title='How the target probability depends on the int features')

In [ ]:
# Check string feature f_27
train['f_27'].value_counts()

In [ ]:
pd.concat([train, test]).f_27.value_counts()

In [ ]:
# Check distribution of eachletter in string

for i in range(10):
    print(f'Position {i}')
    tg = train.groupby(train.f_27.str.get(i))
    temp = pd.DataFrame({'size': tg.size(), 'probability': tg.target.mean().round(2)})
    print(temp)
    print()

In [ ]:
unique_characters = train.f_27.apply(lambda s: len(set(s))).rename('unique_characters')
tg = train.groupby(unique_characters)
temp = pd.DataFrame({'size': tg.size(), 'probability': tg.target.mean().round(2)})
print(temp)


In [ ]:
# Separate string in to 10 unique features
for df in [train, test]:
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))

In [ ]:
plot_mutual_info_diagram(train, 
                         [f for f in train.columns if f.startswith('ch')] + ['unique_characters'],
                         title='How the target probability depends on the character features')

In [ ]:
from matplotlib.colors import ListedColormap

plt.rcParams['axes.facecolor'] = 'k'
plt.figure(figsize=(11, 5))
cmap = ListedColormap(["#ffd700", "#0057b8"])
# target == 0 → yellow; target == 1 → blue

ax = plt.subplot(1, 3, 1)
ax.scatter(train['f_02'], train['f_21'], s=1,
           c=train.target, cmap=cmap)
ax.set_xlabel('f_02')
ax.set_ylabel('f_21')
ax.set_aspect('equal')
ax0 = ax

ax = plt.subplot(1, 3, 2, sharex=ax0, sharey=ax0)
ax.scatter(train['f_05'], train['f_22'], s=1,
           c=train.target, cmap=cmap)
ax.set_xlabel('f_05')
ax.set_ylabel('f_22')
ax.set_aspect('equal')

ax = plt.subplot(1, 3, 3, sharex=ax0, sharey=ax0)
ax.scatter(train['f_00'] + train['f_01'], train['f_26'], s=1,
           c=train.target, cmap=cmap)
ax.set_xlabel('f_00 + f_01')
ax.set_ylabel('f_26')
ax.set_aspect('equal')

plt.tight_layout(w_pad=1.0)
plt.savefig('three-projections.png')
plt.show()
plt.rcParams['axes.facecolor'] = '#0057b8' # blue

In [ ]:
for df in [train, test]:
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)#

In [ ]:
train.head()

In [ ]:
# Gradient boosting algorithm
scaler = StandardScaler()
y=train['target']
x=train.drop(['target', 'f_27'], axis=1)
test=test.drop([ 'f_27'], axis=1)
x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)
x_test=pd.DataFrame(scaler.transform(test))

y_valid, gbm_val_preds, gbm_test_preds=[],[],[]
cal_true, cal_pred=[],[]
feat_importance=pd.DataFrame(index=x.columns)
k_fold=KFold(n_splits=5,shuffle=True, random_state=21)
for fold, (train_idx,val_idx) in enumerate(k_fold.split(x,y)):

  print("\nFold {}".format(fold+1))
  x_train, y_train =x.iloc[train_idx,:],y[train_idx]
  x_val, y_val=x.iloc[val_idx,:],y[val_idx]
  print("Train shape : {}, {}, Valid shape : {},{}".format(x_train.shape,y_train.shape,x_val.shape,y_val.shape))

  params={'boosting_type':'gbdt',
          'n_estimators': 250,
          'num_leaves': 50,
          'learning_rate':0.1,
          'colsample_bytree':0.9,
          'subsample':0.8,
          'reg_alphs':0.1,
          'objective':'binary',
          'metric':'auc',
          'random_state':21}
  gbm= LGBMClassifier(**params).fit(x_train, y_train, eval_set=[(x_train, y_train),(x_val,y_val)],
                                    verbose=100,
                                    eval_metric=['binary_logloss','auc'])
  gbm_prob =gbm.predict_proba(x_val)[:,1]
  y_valid.append(y_val)
  gbm_val_preds.append(gbm_prob)
  gbm_test_preds.append(gbm.predict_proba(x_test)[:,1])
  feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_

  calibrated_gbm=CalibratedClassifierCV(base_estimator=gbm, cv='prefit')
  cal_fit=calibrated_gbm.fit(x_train,y_train)
  cal_probs =calibrated_gbm.predict_proba(x_val)[:,1]
  prob_true,prob_pred =calibration_curve(y_val, cal_probs,n_bins=10)
  cal_true.append(prob_true)
  cal_pred.append(prob_pred)

  auc_score=roc_auc_score(y_val,gbm_prob)
  print("Validation AUC ={:0.4f}".format(auc_score))
  del x_train, y_train, x_val, y_val
  gc.collect()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
colors=px.colors.qualitative.Prism
color=px.colors.qualitative.Plotly
temp=dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), 
                           height=500, width=1000))
def plot_roc_calibration(y_val, y_prob, mpv_cal, fop_cal):
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=np.linspace(0,1,11), y=np.linspace(0,1,11), 
                             name='Random Chance',mode='lines',
                             line=dict(color="Black", width=1, dash="dot")))
    for i in range(len(y_val)):
        y=y_val[i]
        prob=y_prob[i]
        fpr, tpr, thresh = roc_curve(y, prob)
        roc_auc = auc(fpr,tpr)
        fig.add_trace(go.Scatter(x=fpr, y=tpr, line=dict(color=colors[::-1][i+6], width=3), 
                                 hovertemplate = 'True positive rate = %{y:.3f}, False positive rate = %{x:.3f}',
                                 name='Fold {} AUC = {:.4f}'.format(i+1,roc_auc)))
    fig.update_layout(template=temp, title="Cross-Validation ROC Curves", 
                      hovermode="x unified", width=600,height=500,
                      xaxis_title='False Positive Rate (1 - Specificity)',
                      yaxis_title='True Positive Rate (Sensitivity)',
                      legend=dict(orientation='v', y=.07, x=1, xanchor="right",
                                  bordercolor="black", borderwidth=.5))
    fig.show()
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=np.linspace(0,1,11), y=np.linspace(0,1,11), 
                             name='Perfectly Calibrated',mode='lines',
                             line=dict(color="Black", width=1, dash="dot"),legendgroup=2))
    for i in range(len(mpv_cal)):
        mpv=mpv_cal[i]
        fop=fop_cal[i]
        fig.add_trace(go.Scatter(x=mpv, y=fop, line=dict(color=colors[::-1][i+6], width=3), 
                                 hovertemplate = 'Proportion of Positives = %{y:.3f}, Mean Predicted Probability = %{x:.3f}',
                                 name='Fold {}'.format(i+1),legendgroup=2))
    fig.update_layout(template=temp, title="Probability Calibration Curves", 
                      hovermode="x unified", width=600,height=500,
                      xaxis_title='Mean Predicted Probability',
                      yaxis_title='Proportion of Positives',
                      legend=dict(orientation='v', y=.07, x=1, xanchor="right",
                                  bordercolor="black", borderwidth=.5))
    fig.show()
    
def plot_target_predictions(df):
    plot_df=pd.DataFrame.from_dict({'1':(len(df[df.target>0.5])/len(df.target))*100, 
                                    '0':(len(df[df.target<=0.5])/len(df.target))*100}, 
                                   orient='index', columns=['pct'])
    text=['State {}'.format(i) for i in plot_df.index]
    color,pal=['#38A6A5','#E1B580'],['#88CAC9','#EDD3B3']
    if text[0]=='State 0':
        color,pal=color,pal
    else:
        color,pal=color[::-1],pal[::-1]
    fig=go.Figure()
    fig.add_trace(go.Pie(labels=plot_df.index, values=plot_df.pct, hole=.5, 
                         text=text, sort=False, showlegend=False,
                         marker=dict(colors=pal,line=dict(color=color,width=2)),
                         hovertemplate = "State %{label}: %{value:.2f}%<extra></extra>"))
    fig.update_layout(template=temp, title='Predicted Target Distribution', width=700,
                      uniformtext_minsize=15, uniformtext_mode='hide')
    fig.show()
    
plot_roc_calibration(y_valid, gbm_val_preds, cal_true, cal_pred)

In [ ]:
# Check Feature importances
feat_importance['avg']=feat_importance.mean(axis=1)
feat_impotance=feat_importance.sort_values(by='avg',ascending=True)
pal=sns.color_palette("YlGnBu",52).as_hex()
fig=go.Figure()
for i in range(len(feat_importance.index)):
  fig.add_shape(dict(type='line',y0=i, y1=i,x0=0,x1=feat_importance['avg'][i],line_color=pal[::-1][i],opacity=0.8,line_width=4))
  fig.add_trace(go.Scatter(x=feat_importance['avg'],y=feat_importance.index,mode='markers',marker_color=pal[::-1],marker_size=8,
                           hovertemplate='%{y} Importance = %{x: .0f}<extra></extra>'))
  fig.update_layout(template=temp,title='Feature Importance',
                    xaxis=dict(title='Average Importance, zeroline=False'),
                    yaxis_showgrid=False,height=900,width=800)
  fig.show()

In [ ]:
sub_gbm=submit.copy()
sub_gbm['target']=np.mean(gbm_test_preds,axis=0)
sub_gbm.to_csv("submission.csv",index=False)
plot_target_predictions(sub_gbm)